# 1 - Install

In [1]:
!pip install merlin-models
!pip install nvtabular
!pip install implicit
!pip install cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2 - Imports

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from google.colab import drive
import os
import shutil
import numpy as np
from numpy.ma import count
import cython
from scipy.sparse import coo_matrix, csr_matrix
from tqdm.auto import tqdm

import nvtabular as nvt
from nvtabular.ops import AddMetadata

from merlin.core.dispatch import get_lib
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
from merlin.models.utils.dataset import dataset_to_coo
from merlin.models.implicit import AlternatingLeastSquares

/usr/local/lib/python3.9/dist-packages/merlin/dtypes/mappings/cudf.py:57: UserWarning: cuDF dtype mappings did not load successfully due to an error: No module named 'cudf'
  warn(f"cuDF dtype mappings did not load successfully due to an error: {exc.msg}")
/usr/local/lib/python3.9/dist-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")
/usr/local/lib/python3.9/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# 3 - Método com recall

In [23]:
from numpy import double


class AlternatingLeastSquaresWithRecall(AlternatingLeastSquares):
  def ranking_metrics_at_k_recall(model, train_user_items, test_user_items, K=10,
                        show_progress=True, num_threads=1):
    """ Calculates ranking metrics for a given trained model
    Parameters
    ----------
    model : RecommenderBase
        The fitted recommendation model to test
    train_user_items : csr_matrix
        Sparse matrix of user by item that contains elements that were used
            in training the model
    test_user_items : csr_matrix
        Sparse matrix of user by item that contains withheld elements to
        test on
    K : int
        Number of items to test on
    show_progress : bool, optional
        Whether to show a progress bar
    num_threads : int, optional
        The number of threads to use for testing. Specifying 0 means to default
        to the number of cores on the machine. Note: aside from the ALS and BPR
        models, setting this to more than 1 will likely hurt performance rather than
        help.
    Returns
    -------
    float
        the calculated p@k
    """
    if not isinstance(train_user_items, csr_matrix):
      train_user_items = train_user_items.tocsr()

    if not isinstance(test_user_items, csr_matrix):
      test_user_items = test_user_items.tocsr()

    users = test_user_items.shape[0] 
    items = test_user_items.shape[1]
    u = 0
    i = 0
    batch_idx = 0
    # precision
    relevant = 0.0
    pr_div = 0.0
    total = 0.0 
    total_recall = 0.0
    # map
    mean_ap = 0.0
    ap = 0.0
    # ndcg
    cg = (1.0 / np.log2(np.arange(2, K + 2)))
    cg_sum = np.cumsum(cg)
    ndcg = 0.0 
    idcg = 0
    # auc
    mean_auc = 0.0
    auc = 0.0
    hit = 0.0
    miss = 0.0
    num_pos_items = 0.0
    num_neg_items = 0.0

    test_indptr = test_user_items.indptr
    test_indices = test_user_items.indices

    ids = [[],[]]
    batch = []

    likes = []

    batch_size = 1000
    start_idx = 0

    # get an array of userids that have at least one item in the test set
    to_generate = np.arange(users, dtype="int32")
    to_generate = to_generate[np.ediff1d(test_user_items.indptr) > 0]

    progress = tqdm(total=len(to_generate), disable=not show_progress)
    total_recall = total

    while start_idx < len(to_generate):
      batch = to_generate[start_idx: start_idx + batch_size]
      ids, _ = model.recommend(batch, train_user_items[batch], N=K)
      start_idx += batch_size

      #with nogil:
      for batch_idx in range(len(batch)):
        u = batch[batch_idx]
        likes.clear()
        for i in range(test_indptr[u], test_indptr[u+1]):
          likes.append(test_indices[i])

        pr_div += min(K, count(likes))
        ap = 0
        hit = 0
        miss = 0
        auc = 0
        idcg = cg_sum[min(K, count(likes)) - 1]
        num_pos_items = count(likes)
        num_neg_items = items - num_pos_items
        
        for i in range(K):
          if ((ids[batch_idx, i] in likes) and (likes.index(ids[batch_idx, i]) != count(likes))): #likes.end()
            relevant += 1
            hit += 1
            ap += hit / (i + 1)
            ndcg += cg[i] / idcg
          else:
            miss += 1
            auc += hit
        auc += ((hit + num_pos_items) / 2.0) * (num_neg_items - miss)
        mean_ap += ap / min(K, count(likes))
        mean_auc += auc / (num_pos_items * num_neg_items)
        total += 1
        total_recall += count(likes)

      progress.update(len(batch))

    progress.close()
    # print("precision", relevant / pr_div,
    #       "map", mean_ap / total,
    #       "ndcg", ndcg / total,
    #       "auc", mean_auc / total,
    #       "recall", mean_auc / total_recall    
    #)
    return {
      "precision": relevant / pr_div,
      "map": mean_ap / total,
      "ndcg": ndcg / total,
      "auc": mean_auc / total,
      "recall": mean_auc / total_recall
    }

  def evaluate_recall(self, test_dataset: Dataset, k=10):
        """Evaluates the model

        This function evaluates using a variety of ranking metrics, and returns
        a dictionary of {metric_name: value}.

        Parameters
        ----------
        test_dataset : merlin.io.Dataset
            The validation dataset to evaluate
        k : int
            How many items to return per prediction. By default this method will
            return metrics like 'map@10' , but by increasing k you can generate
            different versions
        """
        test = dataset_to_coo(test_dataset).tocsr()
        ret = AlternatingLeastSquaresWithRecall.ranking_metrics_at_k_recall(
            self.implicit_model,
            self.train_data,
            test,
            K=k,
        )
        return {metric + f"@{k}": value for metric, value in ret.items()} 

# 4 - Diretórios

## 4.1 - Remover diretórios

In [4]:
if os.path.exists('train.parquet'):
  os.remove('train.parquet')
if os.path.exists('test.parquet'):
  os.remove('test.parquet')
shutil.rmtree('train', ignore_errors=True)
shutil.rmtree('test', ignore_errors=True)

## 4.2 - Google Drive


In [5]:
drive.mount(r'/content/drive/')
!ls "/content/drive/MyDrive/Dataset"

Mounted at /content/drive/
SMDI-200UE.csv	SMDI-500E_test.csv   SMDI_original.csv
SMDI-500E.csv	SMDI-500E_train.csv  SMDI_test.csv


# 5 - Preliminaries



In [6]:
COL_USER = "user_id"
COL_ITEM = "item_id"
COL_RATING = "rating"
COL_TIMESTAMP = "timestamp"
COL_PREDICTION = "prediction"
TOP_K = 10
INPUT_DATA_DIR = '/content/drive/MyDrive/Dataset/'
df_lib = get_lib()

# 6 - Dataset

In [7]:
columns = [COL_USER, COL_ITEM, COL_RATING]
train = df_lib.read_csv(os.path.join(INPUT_DATA_DIR, "SMDI-500E_train.csv"), names=columns)
test = df_lib.read_csv(os.path.join(INPUT_DATA_DIR, "SMDI-500E_test.csv"), names=columns)

In [8]:
train.to_parquet("train.parquet")
test.to_parquet("test.parquet")

In [9]:
train_dataset = Dataset("train.parquet", cpu=True)
test_dataset = Dataset("test.parquet", cpu=True)

/usr/local/lib/python3.9/dist-packages/merlin/io/dataset.py:254: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/io/dataset.py:254: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [10]:
user_id = ["user_id"] >> AddMetadata(tags=[Tags.USER_ID, "user_id"]) 
item_id = ["item_id"] >> AddMetadata(tags=[Tags.ITEM, "item_id"])
rating = ["rating"] >> AddMetadata(tags=[Tags.TARGET])

In [11]:
workflow = nvt.Workflow(user_id + item_id + rating)
workflow.fit(train_dataset)
workflow.transform(train_dataset).to_parquet("train")
workflow.transform(test_dataset).to_parquet("test")

/usr/local/lib/python3.9/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/io/dataset.py:254: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.US

In [12]:
train_transformed = Dataset("train/*.parquet", cpu=True)
test_transformed = Dataset("test/*.parquet", cpu=True)

/usr/local/lib/python3.9/dist-packages/merlin/io/dataset.py:254: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/io/dataset.py:254: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.9/dis

# 7 - Treinando

In [48]:
als = AlternatingLeastSquaresWithRecall(
    regularization=0.01, 
    iterations=500, 
    factors=1,
    use_gpu=False
  )
als.fit(train_transformed)

  0%|          | 0/500 [00:00<?, ?it/s]

# 8 - Avaliando

In [49]:
implicit_metrics = als.evaluate_recall(test_transformed)
#implicit_metrics['precision@10']
print(str(implicit_metrics['precision@10']).replace('.',','))
print(str(implicit_metrics['map@10']).replace('.',','))
print(str(implicit_metrics['ndcg@10']).replace('.',','))
print(str(implicit_metrics['auc@10']).replace('.',','))
print(str(implicit_metrics['recall@10']).replace('.',','))

  0%|          | 0/3052 [00:00<?, ?it/s]

0,16616973921807415
0,08123073025688593
0,159711217046873
0,5449862401002272
0,033964958950927965


In [35]:
str(implicit_metrics['precision@10']).replace('.',',')
str(implicit_metrics['precision@10']).replace('.',',')
str(implicit_metrics['precision@10']).replace('.',',')
str(implicit_metrics['precision@10']).replace('.',',')
str(implicit_metrics['precision@10']).replace('.',',')

'0,14122538109436023'

## 8.1 - Recomendações

In [15]:
implicit_preds = als.predict(test_transformed)
implicit_preds

(array([[  55,   64,  132, ..., 2394,   43,   67],
        [  64,  808, 2324, ..., 2197,   27,  124],
        [ 132,  808,   55, ..., 4734, 1335, 2827],
        ...,
        [ 132,  808, 2987, ..., 2826, 4734,  143],
        [  64,   53, 2394, ...,  124,   55,  123],
        [ 132, 1481,  104, ..., 1708, 1335, 4175]], dtype=int32),
 array([[1.0892866 , 0.91634655, 0.8530601 , ..., 0.59813535, 0.5195755 ,
         0.4964165 ],
        [1.1415347 , 1.0167503 , 0.9866061 , ..., 0.4640938 , 0.44567838,
         0.40908796],
        [1.1846325 , 1.0166354 , 0.94950193, ..., 0.61257565, 0.49405628,
         0.42708987],
        ...,
        [1.1472728 , 0.8665628 , 0.56467664, ..., 0.23135678, 0.2275287 ,
         0.21766175],
        [1.1701132 , 0.88943326, 0.59647113, ..., 0.31736323, 0.25851795,
         0.20945928],
        [0.9360261 , 0.26666456, 0.18899934, ..., 0.13405505, 0.12040887,
         0.10470951]], dtype=float32))

## 8.2 - Teste usuário 17

In [16]:
filter = [("user_id", "in", [17])]
user_predicted = als.predict(Dataset("test/*.parquet", cpu=True, filters=filter))

/usr/local/lib/python3.9/dist-packages/merlin/io/dataset.py:254: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [17]:
user_predicted[0]

array([[  64,  123,   53, 2324, 2394,  132,   27,   49,   65, 2197]],
      dtype=int32)

In [18]:
user_predicted[1]

array([[1.235002  , 1.1161202 , 1.0595841 , 0.85338044, 0.769241  ,
        0.7003355 , 0.6938758 , 0.6325807 , 0.5475798 , 0.52864856]],
      dtype=float32)

In [19]:
import pandas as pd
pd.DataFrame({'user_id': 17, 'item_id': user_predicted[0][0], 'rating' : user_predicted[1][0]})

,user_id,item_id,rating
0,17,64,1.235002
1,17,123,1.116120
2,17,53,1.059584
3,17,2324,0.853380
4,17,2394,0.769241
5,17,132,0.700336
6,17,27,0.693876
7,17,49,0.632581
8,17,65,0.547580
9,17,2197,0.528649
